# CrewAI Markdown Validator


In [1]:
import sys
from crewai import Agent, Task
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import agentops
from langchain_groq import ChatGroq
from langchain.tools import tool
from pymarkdown.api import PyMarkdownApi, PyMarkdownApiException

In [2]:
load_dotenv()
agentops.init(api_key=os.getenv('AGENTOPS_API_KEY', None), default_tags=['markdown_validator'])

🖇 AgentOps:  WARNING: agentops is out of date. Please update with the command: 'pip install --upgrade agentops'
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=abb92bd6-f875-484e-8fee-318d35f3a936


In [3]:
@tool("markdown_validation_tool")
def markdown_validation_tool(file_path: str) -> str:
    """
    A tool to review files for markdown syntax errors.

    Parameters:
    - file_path: The path to the markdown file to be reviewed.

    Returns:
    - validation_results: A list of validation results 
    and suggestions on how to fix them.
    """

    print("\n\nValidating Markdown syntax...\n\n" + file_path)

    try:
        if not (os.path.exists(file_path)):
            return "Could not validate file. The provided file path does not exist."

        scan_result = PyMarkdownApi().scan_path(file_path.rstrip().lstrip())
        results = str(scan_result)
        return results  # Return the reviewed document
    except PyMarkdownApiException as this_exception:
        print(f"API Exception: {this_exception}", file=sys.stderr)
        return f"API Exception: {str(this_exception)}"


In [10]:
def process_markdown_document(filename):
    """
    Processes a markdown document by reviewing its syntax validation 
    results and providing feedback on necessary changes.

    Args:
        filename (str): The path to the markdown file to be processed.

    Returns:
        str: The list of recommended changes to make to the document.

    """

    # Define general agent
    general_agent  = Agent(role='Requirements Manager',
                    goal="""Provide a detailed list of the markdown 
                            linting results. Give a summary with actionable 
                            tasks to address the validation results. Write your 
                            response as if you were handing it to a developer 
                            to fix the issues.
                            DO NOT provide examples of how to fix the issues or
                            recommend other tools to use.""",
                    backstory="""You are an expert business analyst 
					and software QA specialist. You provide high quality, 
                    thorough, insightful and actionable feedback via 
                    detailed list of changes and actionable tasks.""",
                    allow_delegation=False, 
                    verbose=True,
                    tools=[markdown_validation_tool],
                    llm=default_llm)  #groq_llm) #


    # Define Tasks Using Crew Tools
    syntax_review_task = Task(description=f"""
			Use the markdown_validation_tool to review 
			the file(s) at this path: {filename}
            
			Be sure to pass only the file path to the markdown_validation_tool.
			Use the following format to call the markdown_validation_tool:
			Do I need to use a tool? Yes
			Action: markdown_validation_tool
			Action Input: {filename}

			Get the validation results from the tool 
			and then summarize it into a list of changes
			the developer should make to the document.
            DO NOT recommend ways to update the document.
            DO NOT change any of the content of the document or
            add content to it. It is critical to your task to
            only respond with a list of changes.
			
			If you already know the answer or if you do not need 
			to use a tool, return it as your Final Answer.""",
            agent=general_agent,
            expected_output="")
    
    updated_markdown = syntax_review_task.execute_sync()

    return updated_markdown

In [11]:
groq_llm = ChatGroq(
            temperature=0,
            groq_api_key=os.environ.get("GROQ_API_KEY"),
            model_name="llama3-70b-8192",
        )

default_llm = ChatOpenAI(openai_api_base=os.environ.get("OPENAI_API_BASE_URL", "https://api.openai.com/v1"),
                        openai_api_key=os.environ.get("OPENAI_API_KEY"),
                        temperature=0.1,                        
                        model_name=os.environ.get("MODEL_NAME", "gpt-3.5-turbo"))


In [12]:
filename = "README.md"
processed_document = process_markdown_document(filename)
print(processed_document)



> Entering new CrewAgentExecutor chain...
I need to use the markdown_validation_tool to review the README.md file and provide a list of changes for the developer to address.

Action: markdown_validation_tool
Action Input: {'file_path': 'README.md'}


Validating Markdown syntax...

README.md
 

PyMarkdownScanPathResult(scan_failures=[PyMarkdownScanFailure(scan_file='README.md', line_number=3, column_number=1, rule_id='MD022', rule_name='blanks-around-headings,blanks-around-headers', rule_description='Headings should be surrounded by blank lines.', extra_error_information=' [Expected: 1; Actual: 0; Below]'), PyMarkdownScanFailure(scan_file='README.md', line_number=4, column_number=1, rule_id='MD013', rule_name='line-length', rule_description='Line length', extra_error_information=' [Expected: 80, Actual: 288]'), PyMarkdownScanFailure(scan_file='README.md', line_number=6, column_number=1, rule_id='MD022', rule_name='blanks-around-headings,blanks-around-headers', rule_description='Headin

In [13]:
agentops.end_session('Success')

🖇 AgentOps: This run's cost $0.002789
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=abb92bd6-f875-484e-8fee-318d35f3a936
